## Первая генеративная модель

Использую датасет CIFAR-10

##### Импорт модулей

In [2]:
import numpy as np
from keras.utils import to_categorical
from keras.datasets import cifar10

##### Загрузка датасета

Данные x_train представлены в виде вектора (50000, 32, 32, 3)

Данные y_train в виде целочисленной метки от 0 до 9, их преобразуем в вектор длины NUM_CLASSES, где NUM_CLASSES - количество категорий. В получившимся векторе для изображения категории i нулю равны все элементы кроме элемента с индексом i

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
NUM_CLASSES = 10 # 10 категорий изображений в cifar-10
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

##### Конструирование модели

In [10]:
from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model

###### Исходный вариант

In [5]:
input_layer = Input(shape=(32, 32, 3)) # форма слоя - форма элемента из датасета
x = Flatten()(input_layer) # преобразуем данные в вектор с помощью слоя Flatten. В результате получается вектор с длиной 3072 (= 32 × 32 × 3). Следующий слой потребует данные в виде вектора
x = Dense(units=200, activation='relu')(x) # Полносвязный слой, с функцией активации
x = Dense(units=150, activation='relu')(x) # Полносвязный слой, с функцией активации
output_layer = Dense(units=10, activation='softmax')(x) # Выходной слой с функцией softmax
model = Model(input_layer, output_layer) # Модель, определяемая входными и выходными слоями, в данном случае 1 входной и 1 выходной

###### Структура с четырьмя сверточными слоями

In [11]:
input_layer = Input((32,32,3))

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Flatten()(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)
output_layer = Dense(units=NUM_CLASSES, activation='softmax')(x)
model = Model(input_layer, output_layer)

##### Информация о сети

In [12]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 32)        128       
 chNormalization)                                          

##### Компиляция модели

In [13]:
from keras.optimizers import Adam

In [14]:
opt = Adam(learning_rate=0.0005)

In [15]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics='accuracy')

##### Обучение модели

In [16]:
model.fit(x_train, y_train, batch_size=32, epochs=10, shuffle=True)

Epoch 1/10
1563/1563 [==============================] - 74s 46ms/step - loss: 1.5423 - accuracy: 0.4610
Epoch 2/10
1563/1563 [==============================] - 73s 47ms/step - loss: 1.1298 - accuracy: 0.6041
Epoch 3/10
1563/1563 [==============================] - 73s 47ms/step - loss: 0.9941 - accuracy: 0.6516
Epoch 4/10
1563/1563 [==============================] - 74s 48ms/step - loss: 0.9136 - accuracy: 0.6825
Epoch 5/10
1563/1563 [==============================] - 75s 48ms/step - loss: 0.8566 - accuracy: 0.7014
Epoch 6/10
1563/1563 [==============================] - 76s 49ms/step - loss: 0.8010 - accuracy: 0.7203
Epoch 7/10
1563/1563 [==============================] - 74s 47ms/step - loss: 0.7595 - accuracy: 0.7336
Epoch 8/10
1563/1563 [==============================] - 75s 48ms/step - loss: 0.7264 - accuracy: 0.7465
Epoch 9/10
1563/1563 [==============================] - 75s 48ms/step - loss: 0.6855 - accuracy: 0.7605
Epoch 10/10
1563/1563 [==============================] - 76s 49m

##### Тестирование модели на данных, не использовавшихся для обучения сети

In [17]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 4s 13ms/step - loss: 0.8220 - accuracy: 0.7175


[0.8219518661499023, 0.7174999713897705]